# Rens' his place to play around in.

The contruction of DIS

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import DarkNews as dn


import matplotlib.pyplot as plt

In [2]:
# targets
proton = dn.detector.NuclearTarget("H1")
C12    = dn.detector.NuclearTarget("C12")
Ar40   = dn.detector.NuclearTarget("Ar40")

In [11]:
import parton as lhapdf #credit LHAPDF

In [7]:
p = lhapdf.mkPDF('CT18NLO',0,pdfdir='../src/DarkNews/include/pdf_data/')

In [24]:
print(dir(lhapdf.__path__))
print(dir(lhapdf.mkPDF.xfxQ2))


['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__siz

In [12]:

    cfg = lhapdf.getConfig()
    if target.is_proton == True: #set LHAPDF metadeta
        cfg.set_entry("Particle", 2212) #PID of target
    elif target.is_neutron == True:
        cfg.set_entry("Particle", 2112)

AttributeError: module 'parton' has no attribute 'getConfig'

In [ ]:
def DIS(Q2, x_bj,process, diagrams=["total"])

    target = process.target

    # masses
    M = target.mass
    Z = target.Z
    mHNL = process.m_ups
    mzprime = process.mzprime
    MSCALAR = process.mhprime
    Enu = process.Enu
    
    Vhad = process.Vhad # Z' int with had
    Vij = process.Vij
    Vji = process.Vji
    lhapdf.setPaths(['include/pdf_data'])
    cfg = lhapdf.getConfig()
    if target.is_proton == True: #set LHAPDF metadeta
        cfg.set_entry("Particle", 2212) #PID of target
    elif target.is_neutron == True:
        cfg.set_entry("Particle", 2112)
    
    del(cfg)
    #PDF reading
    p = lhapdf.mkPDF("CT18NLO", 0) #allow user to choose +def-CT18?
    flavs = p.flavors()[1:-2]#d,u,s,c excluding b and gluon
    f_half = 4
    R =0.3
    R_1 = 1/R #ratio for F_L should be calculated instead
    #zeros = np.zeros(f_half)
    
    g_Z2_V = np.zeros(f_half)
    g_Z2_A = np.zeros(f_half)
    
    g_Z1_V = np.zeros(f_half)
    g_Z1_A = np.zeros(f_half)
    
    g_Z2_V[0] =  g_Z2_V[2] = TheoryModel.ddV #down vectorial
    g_Z2_V[1] = g_Z2_V[3] =TheoryModel.duV #up vectorial
    g_Z2_A[0] =g_Z2_A[2]  = TheoryModel.ddA #down axial
    g_Z2_A[1] =g_Z2_A[3]=  TheoryModel.duA #up axial
    
    g_Z1_V[0] =  g_Z1_V[2] = TheoryModel.cdV #down vectorial
    g_Z1_V[1] = g_Z1_V[3] =TheoryModel.cuV #up vectorial
    g_Z1_A[0] =g_Z1_A[2]  = TheoryModel.cdA #down axial
    g_Z1_A[1] =g_Z1_A[3]=  TheoryModel.cuA #up axial
    
    
    def Lmunu_Hmunu_DC_SQR_DIS():
        xf = np.array(p.xfxQ2(flavs, x, Q2))
        F2_Z2 = np.sum((g_Z2_V**2+g_Z2_A**2) *(xf[f_half:]+xf[:f_half]))
        xF3_Z2 = np.sum((2*g_Z2_V*g_Z2_A )*(xf[f_half:]-xf[:f_half]))
        FL_Z2= (R_1+1)*F2_Z2 
        x2F1_Z2 = F2_Z2*(1+4*M**2 * x**2 /Q2)/(1+FL_Z2)
        y = Q2/( x*2*M*Enu) 
        ds1 = 0.5*y**2 *x2F1_Z2 +(1-y-x*y*M/(2*Enu))*F2_Z2 
        ds2 = y*(1-y*0.5)*xF3_Z2
        anti =1
        ds = (Vij*Vji)*(Vhad*Vhad) *M*Enu/(np.pi*(mzprime**2+Q2)**2) *(ds1+anti*ds2)
        return(ds)#**2?